<a name="title"><h1>Intertidal Mapping</h1></a>

* Jupyter Notebook running on Cloud9
* Using Google Earth Engine for the image analysis
* Author: Andrew Cottam, Joint Research Centre
* Date: July 2017

<a name="contents"><h1>Table of Contents</h1></a>

<ul>
<li><a href=#code>Code</a>
<ul>
<li><a href="#initialisation">Initialisation</a>
<li><a href="#constants">Constants</a>
<li><a href="#areas">Study areas</a>
<li><a href="#functions">Global functions</a>
</ul>
<li><a href=#s1>Sentinel 1 Synthetic Aperture Radar (SAR)</a>
<li><a href=#s2>Sentinel 2 Multi-Spectral Instrument (MSI)</a>
<ul><li><a href="#bands">Bands</a>
</ul>
<li><a href="#examples">Intertidal Examples</a>
<li><a href="#factors">Factors affecting spectral properties</a>
<ul>
<li><a href="#depth">Water depth</a>
<li><a href="#illumination">Solar illumination</a>
<li><a href="#clouds">Clouds</a>
<ul>
<li><a href="#vapor">Water Vapor band (B9)</a>
<li><a href="#cirrus">Cirrus band (B10)</a>
<li><a href="#qa60">QA band (QA60)</a>
</ul>
<li><a href="#shadow">Cloud shadow</a>
<li><a href="#habitats">Habitat types</a>
<li><a href="#condition">Habitat condition</a>
<li><a href="#turbidity">Turbidity</a>
</ul>
<li><a href="#classification">Classification</a>
<ul>
<li><a href=#unsupervised>Unsupervised</a>
<li><a href=#supervised>Supervised</a>
<ul>
</ul>

<a name="code"><h2>Code</h2></a>

<a name="initialisation"><h3>Initialisation</h3></a>

In [5]:
from IPython.display import Image, display, HTML
import ee
ee.Initialize()

<a name="constants"><h3>Constants</h3></a>

In [6]:
SENTINEL1 = ee.ImageCollection('COPERNICUS/S1_GRD')
SENTINEL2 = ee.ImageCollection('COPERNICUS/S2')
GSW = ee.Image("JRC/GSW1_0/GlobalSurfaceWater")
BATHYMETRY = ee.Image("NOAA/NGDC/ETOPO1")
SCALING_FACTOR = 5000
MONTHS = ["Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"]
BANDS = ["B1","B2","B3","B4","B5","B6","B7","B8","B8A","B9","B10","B11","B12"]
UNSUPERVISED_BANDS = ["B2","B3","B4","B5","B6","B7","B8","B8A","B11","B12"] # we dont want to include all bands in the unsupervised classification
BAND_NAMES = ["Coastal aerosol","Blue","Green","Red","Vegetation Red Edge 1","Vegetation Red Edge 2","Vegetation Red Edge 3","NIR","Narrow NIR","Water vapour","SWIR Cirrus","SWIR 1","SWIR 2"]
BAND_RESOLUTIONS = [60,10,10,10,20,20,20,10,20,60,60,20,20]

<a name="areas"><h3>Study areas</h3></a>

In [7]:
# scale is the distance across the image in Kilometers
AREA_CHALK_SOUND = {'centroid': ee.Geometry.Point(-72.29, 21.77), 'scale': 10}
AREA_TURKS_AND_CAICOS = {'centroid': ee.Geometry.Point(-72.1, 21.7), 'scale': 60}
BOTTLE_CREEK = {'centroid': ee.Geometry.Point(-71.8923, 21.88444), 'scale': 10}
BIG_POND = {'centroid': ee.Geometry.Point(-71.70021, 21.7595), 'scale': 20}
BIG_POND2 = {'centroid': ee.Geometry.Point(-71.70021, 21.7595), 'scale': 5}

<a name="functions"><h3>Global functions</h3></a>

In [8]:
#returns an extent object using the passed centroid coordinate and the desired width in kilometers
def getExtent(centroid, width):
    width = ee.Number(width).multiply(1000)
    scale = centroid.projection().nominalScale()
    widthLL = ee.Number(width).divide(scale).divide(2)
    heightLL = ee.Number(width).divide(scale).divide(2)
    llx = ee.Number(centroid.coordinates().get(0)).subtract(widthLL)
    lly = ee.Number(centroid.coordinates().get(1)).subtract(heightLL)
    urx = ee.Number(centroid.coordinates().get(0)).add(widthLL)
    ury = ee.Number(centroid.coordinates().get(1)).add(heightLL)
    return ee.Geometry.Rectangle([llx,lly,urx,ury], None, False)

# converts the image to hsv
def getHsv(image):
    return image.select(["B4", "B3", "B2"]).divide(SCALING_FACTOR).rgbToHsv()

# gets the scenes for a specific area for a specific month for sentinel 2
def getMonthImage(month):
    #filter the data by month and study area
    monthIndex = ee.Number(ee.List(MONTHS).indexOf(month)).add(1);
    scenes = SENTINEL2.filterBounds(study_area).filter(ee.Filter.calendarRange(monthIndex,monthIndex,"month")).sort("CLOUDY_PIXEL_PERCENTAGE", False) # filter for the study area and the month and order with the least cloudy images on top
    #mosaic the data
    mosaic = scenes.mosaic()
#     mosaic = scenes.reduce(ee.Reducer.percentile([30])).rename(["B1","B2","B3","B4","B5","B6","B7","B8","B8A","B9","B10","B11","B12","QA10","QA20","QA60"])
#     mosaic = scenes.reduce(ee.Reducer.mean()).rename(["B1","B2","B3","B4","B5","B6","B7","B8","B8A","B9","B10","B11","B12","QA10","QA20","QA60"])
#     mosaic = ee.Algorithms.If(ee.String(_bands).index("hue").eq(0).Or(ee.String(_bands).index("saturation").eq(0)).Or(ee.String(_bands).index("value").eq(0)), getHsv(mosaic), mosaic)
    return ee.Image(mosaic).copyProperties(ee.Image(scenes.first())) #copy the metadata properties of the first (arbritrary) image to the mosaic

# gets the clouds from the sentinel qa60 band
def getCloudsQA60(image):
    image = image.select('QA60').divide(ee.Number(2).pow(10)) #bits 10 and 11 denote cloud and cirrus respectively so output will be 0=no cloud, 1=cloud, 2=cirrus, 4=both
    image = image.mask(image)
    return image

# gets the scenes for a specific area for a specific month for sentinel 1
def getMonthImageS1(month):
    #filter the data by month and study area
    monthIndex = ee.Number(ee.List(MONTHS).indexOf(month)).add(1);
    scenes = SENTINEL1.filterBounds(study_area).filter(ee.Filter.calendarRange(monthIndex,monthIndex,"month")) # filter for the study area and the month 
    #mosaic the data
    mosaic = scenes.mean()
    return ee.Image(mosaic).copyProperties(ee.Image(scenes.first())) #copy the metadata properties of the first (arbritrary) image to the mosaic

# gets the study area
def getStudyArea(area):
    #get the extent of the passed area
    global study_area
    study_area = getExtent(area["centroid"], area["scale"])
    return study_area

# shows a single image 
def getThumbUrl(image, area, bands, size=300, min=0, max=5000):
    study_area = getStudyArea(area)
    url = image.getThumbUrl({'region': study_area.getInfo(), 'bands': bands,'min': min, 'max': max, 'dimensions': size})
    return url

    # gets urls for sentinel images for the area for the specific month
def showImages(area, months, bands, size=300):
    #get the study area
    study_area = getStudyArea(area)
    #convert the months from a client side to server side object and get the images
    images = ee.List(months).map(getMonthImage)
    urls = []
    #get the values for the max depending on what bands we want
    if (bands in ["hue","saturation","value"]):
        max = 1
    else:
        max = SCALING_FACTOR
    #get the thumbnail urls for each monthly image
    for i in range(images.size().getInfo()):
        # get the ith monthlyimage
        img = ee.Image(ee.ImageCollection(images).toList(1,i).get(0))
        _url = img.getThumbUrl({'region': study_area.getInfo(), 'bands': bands,'max': max, 'dimensions': size})
        urls.append({'month': months[i],'url': _url, 'zenith': img.get("MEAN_SOLAR_ZENITH_ANGLE").getInfo()})
    #get the html to write to the output cell
    display(HTML(data="""<style>div#notebook-container{width:95%;}div#menubar-container{width:65%;}div#maintoolbar-container{width:99%;}</style>"""))
    html= "<table><tr>"
    for i in range(len(urls)):
        image = urls[i]
        if (i%4==0):
            html += "</tr><tr>"
        html += "<td><div><div style='float:left'><img src='" + image["url"] + "'></div><div style='font-size:24px;position:absolute;color:white;padding:10px'>" + image["month"] + "</div><div>MEAN_SOLAR_ZENITH_ANGLE: " + str(int(image["zenith"])) + "</div></div></td>"
    html += "</tr></table>"
    #write the output cell
    display(HTML(html))
    
# gets the urls for the sentinel images for each band
def showBands(area, month, bands, stretches={}, size=300):
    # get the study area
    study_area = getStudyArea(area)
    # get the mosaic image
    image = getMonthImage(month)
    #get the thumbnail urls for each band
    urls = []
    for i in range(len(bands)):
        band = bands[i]
        bandIndex = BANDS.index(band) 
        bandname = BAND_NAMES[bandIndex]
        bandResolution = BAND_RESOLUTIONS[bandIndex]
        if band in stretches.keys(): # if the min/max values for specific bands have been passed then use them
            min = stretches[band][0]
            max = stretches[band][1]
        else:
            min = 0
            max = SCALING_FACTOR
        _url = ee.Image(image).getThumbUrl({'region': study_area.getInfo(), 'bands': band,'min': min, 'max': max, 'dimensions': size})
        urls.append({'band': band + " " + bandname + " (" + str(bandResolution) + "m)", 'url': _url})
    #get the html to write to the output cell
    display(HTML(data="""<style>div#notebook-container{width:95%;}div#menubar-container{width:65%;}div#maintoolbar-container{width:99%;}</style>"""))
    html= "<table><tr>"
    for i in range(len(urls)):
        image = urls[i]
        if (i%4==0):
            html += "</tr><tr>"
        html += "<td><div><div style='float:left'><img src='" + image["url"] + "'></div><div style='font-size:20px;position:absolute;color:white;padding:10px'>" + image["band"] + "</div></div></td>"
    html += "</tr></table>"
    #write the output cell
    display(HTML(html))

# gets the sentinel 1 images
def showImagesS1(area, months, band, size=300):
    #get the study area
    study_area = getStudyArea(area)
    #convert the months from a client side to server side object and get the images
    images = ee.List(months).map(getMonthImageS1)
    urls = []
    #get the thumbnail urls for each monthly image
    for i in range(images.size().getInfo()):
        # get the ith monthlyimage
        img = ee.Image(ee.ImageCollection(images).toList(1,i).get(0))
        _url = img.getThumbUrl({'region': study_area.getInfo(), 'bands': band,'min': -20, 'max': 0, 'dimensions': size})
        urls.append({'month': months[i],'url': _url})
    #get the html to write to the output cell
    display(HTML(data="""<style>div#notebook-container{width:95%;}div#menubar-container{width:65%;}div#maintoolbar-container{width:99%;}</style>"""))
    html= "<table><tr>"
    for i in range(len(urls)):
        image = urls[i]
        if (i%4==0):
            html += "</tr><tr>"
        html += "<td><div><div style='float:left'><img src='" + image["url"] + "'></div><div style='font-size:24px;position:absolute;color:white;padding:10px'>" + image["month"] + "</div></div></td>"
    html += "</tr></table>"
    #write the output cell
    display(HTML(html))
    
def unsupervisedClassification(area, classes, size=300):
    #get the study area
    study_area = getStudyArea(area)
    #get the maximum water extent
    max_extent = GSW.select(["max_extent"]).updateMask(BATHYMETRY.select(["ice_surface"]).gte(-750))
    #get the input imagery
    s2c = SENTINEL2.filterBounds(study_area)
    #reduce the amount of cloud in the composite
    s2_all = s2c.reduce(ee.Reducer.percentile([30])).rename(["B1","B2","B3","B4","B5","B6","B7","B8","B8A","B9","B10","B11","B12","QA10","QA20","QA60"])
    #mask with the high water mark
    s2_masked = s2_all.mask(max_extent)
    #train on some pixels
    training = s2_masked.select(UNSUPERVISED_BANDS).sample(study_area.getInfo(),30)
    #classify using the weka means clusterer
    classified = s2_masked.cluster(ee.Clusterer.wekaKMeans(classes).train(training)).randomVisualizer()
    url = classified.getThumbUrl({'region': study_area.getInfo(), 'dimensions': size})
    html = "<img src='" + url + "'>"
    display(HTML(html)) 

<a name="s1"><h2>Sentinel 1 Synthetic Aperture Radar (SAR)</h2>

<a name="bands"><h3>Bands</h3></a>

There are four polarisations on the SAR sensor (VV+VH,HH+HV,HH,VV) but they are exclusive.

The following images show the mean backscatter for January and June.

In [10]:
showImagesS1(AREA_CHALK_SOUND, ["Jan","Jul"], "VV", 800)

Jan,Jul


<a name="s2"><h2>Sentinel 2 Multi-Spectral Instrument (MSI)</h2>

<a name="bands"><h3>Bands</h3></a>

The following image shows the distribution of bands in the Sentinel 2 sensor compared with Landsat sensors and the technical specification for the bands.

<table style="border:0px;margin-left:0px"><tr style="border:0px"><td><img src='https://landsat.gsfc.nasa.gov/wp-content/uploads/2015/06/Landsat.v.Sentinel-2.png' style='width:900px;margin-left:0px'></td><td>
<table class="wikitable" style="border:0px;margin-left:0px;font-size:0.85em">
<tr style="border:0px">
<th>Sentinel-2 Bands</th>
<th>Central Wavelength (µm)</th>
<th>Resolution (m)</th>
</tr>
<tr>
<td>Band 1 – Coastal aerosol</td>
<td>0.443</td>
<td>60</td>
</tr>
<tr>
<td>Band 2 – Blue</td>
<td>0.490</td>
<td>10</td>
</tr>
<tr>
<td>Band 3 – Green</td>
<td>0.560</td>
<td>10</td>
</tr>
<tr>
<td>Band 4 – Red</td>
<td>0.665</td>
<td>10</td>
</tr>
<tr>
<td>Band 5 – Vegetation Red Edge</td>
<td>0.705</td>
<td>20</td>
</tr>
<tr>
<td>Band 6 – Vegetation Red Edge</td>
<td>0.740</td>
<td>20</td>
</tr>
<tr>
<td>Band 7 – Vegetation Red Edge</td>
<td>0.783</td>
<td>20</td>
</tr>
<tr>
<td>Band 8 – NIR</td>
<td>0.842</td>
<td>10</td>
</tr>
<tr>
<td>Band 8A – Narrow NIR</td>
<td>0.865</td>
<td>20</td>
</tr>
<tr>
<td>Band 9 – Water vapour</td>
<td>0.945</td>
<td>60</td>
</tr>
<tr>
<td>Band 10 – SWIR – Cirrus</td>
<td>1.375</td>
<td>60</td>
</tr>
<tr>
<td>Band 11 – SWIR</td>
<td>1.610</td>
<td>20</td>
</tr>
<tr>
<td>Band 12 – SWIR</td>
<td>2.190</td>
<td>20</td>
</tr>
</table></td></tr></table>

The images below show all of the band images for the Turks and Caicos.

In [11]:
showBands(AREA_CHALK_SOUND, "Jan", BANDS, {"B10":[0,100]})
# showBands(AREA_CHALK_SOUND, "Jan", ["B1"])
# showBands(AREA_CHALK_SOUND, "Jan", BANDS, {"B1":[1000,4000],"B4":[0,4000],"B6":[0,1000],"B7":[0,1000],"B8":[0,1000],"B8A":[0,1000],"B9":[0,1000],"B10":[0,1000],"B11":[0,1000]}, 400)

<a name="examples"><h2>Intertidal Examples</h2></a>

An example mosaic image for the Turks and Caicos Islands for January is shown below:

In [8]:
showImages(AREA_CHALK_SOUND, ["Jan"], "B4,B3,B2", 800)

JanMEAN_SOLAR_ZENITH_ANGLE: 49


An example for a single band (in this case the blue band) is shown below.

<a name="factors"><h2>Factors affecting spectral properties</h2></a>

Changes in the surface spectral properties are due to differences in solar illumination, cloud contamination, habitat types, conditions and seasonality. They are also due to differences in the physical properties of the water, e.g. to the turbidity. 

<a name='depth'><h3>Water depth</h3></a>

The spectral properties will be affected by the depth of the water, including whether water is present or not for intertidal zones. This will depend on the height of the tide.

<a name="illumination"><h3>Solar illumination</h3></a>

The following example shows the variability in the illumination of the surface.

In [9]:
# showImages(AREA_CHALK_SOUND, ["Jan","Aug"], "B4,B3,B2", 600)
showImages(AREA_CHALK_SOUND, MONTHS, "B4,B3,B2", 300)

JanMEAN_SOLAR_ZENITH_ANGLE: 49,FebMEAN_SOLAR_ZENITH_ANGLE: 43,MarMEAN_SOLAR_ZENITH_ANGLE: 29,AprMEAN_SOLAR_ZENITH_ANGLE: 18
MayMEAN_SOLAR_ZENITH_ANGLE: 16,JunMEAN_SOLAR_ZENITH_ANGLE: 19,JulMEAN_SOLAR_ZENITH_ANGLE: 21,AugMEAN_SOLAR_ZENITH_ANGLE: 22
SepMEAN_SOLAR_ZENITH_ANGLE: 28,OctMEAN_SOLAR_ZENITH_ANGLE: 38,NovMEAN_SOLAR_ZENITH_ANGLE: 43,DecMEAN_SOLAR_ZENITH_ANGLE: 48


In [10]:
showImages(AREA_CHALK_SOUND, ["Jan","Aug"], "hue", 600)

EEException: Image.visualize: No band named 'hue'. Available band names: [B1, B2, B3, B4, B5, B6, B7, B8, B8A, B9, B10, B11, B12, QA10, QA20, QA60].

In [11]:
showImages(AREA_CHALK_SOUND, ["Jan","Aug"], "saturation", 600)

EEException: Image.visualize: No band named 'saturation'. Available band names: [B1, B2, B3, B4, B5, B6, B7, B8, B8A, B9, B10, B11, B12, QA10, QA20, QA60].

In [ ]:
showImages(AREA_CHALK_SOUND, ["Jan","Aug"], "value", 600)

<a name="clouds"><h3>Clouds</h3></a>

<a name='vapor'><h4>Water vapor (B9)</h4></a>

The Water Vapor band is supposed to show water vapor but it looks like it shows other things.

In [10]:
image = ee.Image("COPERNICUS/S2/20160817T152642_20160817T202520_T19QBE")
url = getThumbUrl(image, BIG_POND, "B4,B3,B2",500)
url2 = getThumbUrl(image, BIG_POND, "B9", 500, 0, 800)
html= "<table><tr><td><img src='" + url + "'></td><td><img src='" + url2 + "'></td></tr></table>"
display(HTML(html)) 

,


<a name='cirrus'><h4>Cirrus band (B10)</h4></a>


The Cirrus band looks really good - here is the data for the above scene. Not sure what the line is going down the image!

In [11]:
image = ee.Image("COPERNICUS/S2/20160817T152642_20160817T202520_T19QBE")
url = getThumbUrl(image, BIG_POND, "B4,B3,B2",500)
url2 = getThumbUrl(image, BIG_POND, "B10", 500, 0, 300)
html= "<table><tr><td><img src='" + url + "'></td><td><img src='" + url2 + "'></td></tr></table>"
display(HTML(html)) 

,


<a name='qa60'><h4>QA band (QA60)</h4></a>

The QA60 band holds information on cumulus and cirrus clouds and this information is held in bits 10 (cumulus) and 11 (cirrus). A full description of the cloud mask is given <a href='https://earth.esa.int/web/sentinel/technical-guides/sentinel-2-msi/level-1c/cloud-masks'>here</a>. The following example shows detection of the cumulus (dark grey) and cirrus (light grey) in a single S2 image. As you can see the cloud detection is not complete.<br/>

In [12]:
image = ee.Image("COPERNICUS/S2/20160817T152642_20160817T202520_T19QBE")
url = getThumbUrl(image, BIG_POND, "B4,B3,B2",500)
url2 = getThumbUrl(getCloudsQA60(image), BIG_POND, "QA60", 500, 0, 4)
html= "<table><tr><td><img src='" + url + "'></td><td><img src='" + url2 + "'></td></tr></table>"
display(HTML(html)) 

,


<a name='shadow'><h3>Cloud shadow</h3></a>

<a name="habitats"><h3>Habitat types</h3></a>

<a name="condition"><h3>Habitat condition</h3></a>

<a name="turbidity"><h3>Turbidity</h3></a>

<a name="classification"><h2>Habitat Classification</h2></a>

<a name="unsupervised"><h3>Unsupervised</h3></a>

The following image shows an unsupervised classification using the Weka K-means clustering algorithm with 15 classes. The bands that are being used in the unsupervised classification are red, green, blue, vegetation red edge, nir, narrow nir and swir 1 and 2

In [ ]:
unsupervisedClassification(BOTTLE_CREEK, 15, 500)

<a name="supervised"><h3>Supervised</h3></a>